In [13]:
from v2_model import dataCorrection
from v1_model import positivization, map2level, distanceCal, TOPSIS
import pandas as pd
import numpy as np

In [14]:
def sensi_data(path, alpha=0):

    # load the data from csv file and wash them
    file = pd.read_csv(path)
    annual_light = list(file['年光照时间'].astype(int))
    annual_light = positivization(annual_light, mode='1')

    airpollution_list = list(file['空气质量'].astype(float))
    airpollution_list = positivization(airpollution_list)

    popu_list = list(file['人口数量'].astype(int))
    population_list = positivization(popu_list, mode='0', toOne='minmax')
    

    GDP_list = list(file['人均GDP'].astype(str))
    new = []
    for i in GDP_list:
        i = i.replace(',','')
        new.append(int(i))
    GDP_list = positivization(new)

    temp_list = list(file['温度'].astype(str))
    low_list, high_list = [], []
    for tem_tuple in temp_list:
        tem_tuple = tem_tuple.replace('（','')
        tem_tuple = tem_tuple.replace('）','')
        tem_tuple = tem_tuple.split('，')
        low_list.append(int(tem_tuple[0]))
        high_list.append(int(tem_tuple[1]))   
    low_list = positivization(low_list, mode='3', ifStd=False)
    high_list = positivization(high_list, mode='3', ifStd=False)
    new = []
    for k in range(len(low_list)):
        new.append(low_list[k]+high_list[k])
    temp_list = positivization(new)

    rain_list = list(file['降雨量'].astype(int))
    rain_list = positivization(rain_list, mode='2', best_cir= int(sum(rain_list)/len(rain_list)))

    electric_list = list(file['居民用电量'].astype(int))
    electric_list = positivization(electric_list)

    skyglow_list = list(file['skyglow'].astype(int))
    skyglow_list = positivization(skyglow_list)

    UL_intensity_list = list(file['average_light'].astype(float))
    new = UL_intensity_list
    for i in [0,4,8,12]:
        new[i] = new[i] * (1 - alpha)
    UL_intensity_list = positivization(new)

    std_list = list(file['std score'].astype(float))
    std_list = positivization(std_list)

    clutter_list = list(file['clutter score'].astype(float))
    clutter_list = positivization(clutter_list)

    knum_list = list(file['cluster number'].astype(int))
    knum_list = positivization(knum_list)

    Unit_light_intensity_list = dataCorrection(
        target= UL_intensity_list,
        input= [annual_light, airpollution_list, temp_list, rain_list, skyglow_list]
    )

    Development_list = dataCorrection(
        target= GDP_list,
        input= [population_list, electric_list]
    )

    Clutter_list = dataCorrection(
        target= clutter_list,
        input=[std_list, knum_list]
    )

    return Unit_light_intensity_list, Development_list, Clutter_list

In [15]:
file = pd.read_csv('./collect.csv')
name_list = list(file['地点'])

for i in range(5):
    alpha = (1+i)*0.05
    all = np.array(sensi_data('./collect.csv', alpha))
    weight = distanceCal(all, aug_pos=-1)
    dis_weight = TOPSIS(all, weight)
    map2level(all, dis_weight, name_list, save_path='./new.csv')

Name: Paris   
predict score:8

Name: Los Angeles   
predict score:9

Name: London   
predict score:7

Name: Barcelona   
predict score:7

Name: Versailles(Paris)   
predict score:5

Name: Anaheim(LA)   
predict score:7

Name: Stonecrest(Atlanta)   
predict score:5

Name: Ann Arbor(Detroit)   
predict score:5

Name: Millau   
predict score:2

Name: Vicksburg   
predict score:2

Name: Montalcino   
predict score:2

Name: Grindelwald   
predict score:4

Name: Yellowstone N.P.   
predict score:2

Name: Berchtesgaden N.P.   
predict score:1

Name: Grand Canyon N.P.   
predict score:2

Name: Serengeti N.P.   
predict score:2

Name: Paris   
predict score:8

Name: Los Angeles   
predict score:9

Name: London   
predict score:7

Name: Barcelona   
predict score:7

Name: Versailles(Paris)   
predict score:5

Name: Anaheim(LA)   
predict score:7

Name: Stonecrest(Atlanta)   
predict score:5

Name: Ann Arbor(Detroit)   
predict score:5

Name: Millau   
predict score:2

Name: Vicksburg   
predict